In [16]:
import gym
import random
import sys

In [17]:
env = gym.make('SpaceInvaders-v4', render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [18]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [5]:
episodes = 1;
for episode in range(episodes):
    done = False
    state = env.reset()
    score = 0
    while not done:
#         env.render()
        action = random.choice(list(range(6)))
        newState, reward, done, info1, info2 = env.step(action)
#         x = env.step(action)
#         print()
#         sys.exit(0)
        score += reward
        
    print('Episode: {}, Score: {}'.format(episode+1, score))

env.close()

Episode: 1, Score: 290.0


In [19]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [20]:
def build_model(height, width, channels, actions):
    model = Sequential()
    print("hello")
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(3, height, width, channels))) # The first argument is the number of images that you want to pass together
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), activation='relu')) # default stride is (1, 1)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [21]:
model = build_model(height, width, channels, actions)

hello


In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_1 (Flatten)         (None, 67584)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               34603520  
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 6)                

In [23]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [27]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1.0, value_min=0.1, value_test=0.2, nb_steps=1000)
    memory = SequentialMemory(limit = 1000, window_length=3)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, 
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=500)
    return dqn

In [28]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))
dqn.fit(env, nb_steps=1000, visualize=False, verbose=2)

Training for 1000 steps ...


AttributeError: 'int' object has no attribute 'shape'